In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:

pz.ts.active_autovisualizer.set_interactive(pz.ts.ArrayAutovisualizer())

In [3]:
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import random
from penzai.data_effects.side_output import SideOutputValue
from micrlhf.utils.activation_manipulation import add_vector

In [4]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")
from micrlhf.sampling import sample
from transformers import AutoTokenizer
import jax
# tokenizer = load_tokenizer(filename)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [6]:
from task_vector_utils import load_tasks, ICLDataset, ICLSequence
tasks = load_tasks()

Cloning into 'itv'...
fatal: unable to access 'https://github.com/roeehendel/icl_task_vectors data/itv/': URL using bad/illegal format or missing URL


In [7]:
tasks.keys()

dict_keys(['location_continent', 'football_player_position', 'location_religion', 'location_language', 'person_profession', 'location_country', 'country_capital', 'person_language', 'singular_plural', 'present_simple_past_simple', 'antonyms', 'plural_singular', 'present_simple_past_perfect', 'present_simple_gerund', 'en_it', 'it_en', 'en_ru', 'en_fr', 'en_es', 'fr_en', 'es_en', 'en_de', 'algo_max', 'algo_min', 'algo_last', 'algo_first', 'algo_sum', 'algo_most_common'])

In [8]:
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper
get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)

In [9]:
def generate_task_prompt(task, n_shots):
    prompt = "<user>Follow the pattern\n{}"
    examples = []

    for s, t in random.sample(list(tasks[task].items()), n_shots):
        examples.append(f"{s} -> {t}")
    prompt = prompt.format("\n".join(examples))

    # print(prompt)

    return prompt

def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs

In [10]:
def generate_task_inputs_old(task, n_shots, batch_size, max_length=128, seed=0):
    random.seed(seed)

    texts = [generate_task_prompt(task, n_shots) for _ in range(batch_size)]
    tokenized = tokenizer.batch_encode_plus(texts, padding="longest", max_length=max_length, truncation=True, return_tensors="np")

    inputs = tokenized_to_inputs(
        **tokenized
    )

    return inputs, tokenized

In [11]:
prompt = "<user>Follow the pattern\n{}"

In [12]:
def get_logprob_diff(logits: jnp.ndarray, completions: list[str], print_results=False, extra_space=False):
    logprobs = jax.nn.log_softmax(logits, axis=-1)
    answer_logprobs = logprobs[:, -1]

    pos = 1
    if extra_space:
        pos += 1

    target_tokens = [x[pos] for x in tokenizer(completions)["input_ids"]]
    target_tokens = jnp.asarray(target_tokens)
    target_logprobs = jnp.take_along_axis(answer_logprobs, target_tokens[:, None], axis=-1).squeeze()

    if print_results:
        print(
            tokenizer.decode(answer_logprobs.argmax(axis=-1))
        )

        print(
            tokenizer.decode(target_tokens)
        )

    return target_logprobs - answer_logprobs.max(axis=-1)


In [13]:
task_names = [
    "en_it"
]
layer = 20
n_seeds = 10

# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 20, 16, 1024

In [14]:
from task_vector_utils import ICLRunner, logprob_loss, get_tv, make_act_adder


In [291]:

results = {}

task_names = [
    x for x in tasks.keys()
]

for task in tqdm(task_names):
    results[task] = []

    pairs = list(tasks[task].items())

    runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_few_shots-1, max_seq_len=max_seq_len, seed=10)

    tokenized = runner.get_tokens(runner.train_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, resids = get_resids_call(inputs)
    
    tokenized = runner.get_tokens(runner.eval_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    tokens = tokenized["input_ids"]

    logits = llama(inputs)
    

    loss = logprob_loss(
        logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
    )

    print(
        f"T: {task},  zero loss: {loss}"
    )

    results[task].append(loss)

    for layer in trange(10, 28):
        
        # tv = get_tv(resids[layer].value.unwrap("batch", "seq", "embedding"), train_tokens, shift = 1 if task.startswith("algo") else 0)
        tv = get_tv(resids[layer].value.unwrap("batch", "seq", "embedding"), train_tokens, shift = 0)
        tv = tv.astype('bfloat16')

        add_act = make_act_adder(llama, tv, tokens, layer, length=1, shift= 0)

        logits = add_act(inputs)

        loss = logprob_loss(
            logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
        )

        print(
            f"T: {task}, L: {layer}, Loss: {loss}"  
        )

        results[task].append(loss)
        
        

  0%|          | 0/28 [00:00<?, ?it/s]

T: location_continent,  zero loss: 7.96875


  0%|          | 0/18 [00:00<?, ?it/s]

T: location_continent, L: 10, Loss: 8.125
T: location_continent, L: 11, Loss: 8.3125
T: location_continent, L: 12, Loss: 8.125
T: location_continent, L: 13, Loss: 7.46875
T: location_continent, L: 14, Loss: 6.8125
T: location_continent, L: 15, Loss: 6.21875
T: location_continent, L: 16, Loss: 6.15625
T: location_continent, L: 17, Loss: 5.96875
T: location_continent, L: 18, Loss: 5.3125
T: location_continent, L: 19, Loss: 4.96875
T: location_continent, L: 20, Loss: 4.09375
T: location_continent, L: 21, Loss: 3.3125
T: location_continent, L: 22, Loss: 3.23438
T: location_continent, L: 23, Loss: 2.85938
T: location_continent, L: 24, Loss: 2.10938
T: location_continent, L: 25, Loss: 1.91406
T: location_continent, L: 26, Loss: 2.21875
T: location_continent, L: 27, Loss: 2.04688
T: football_player_position,  zero loss: 13.125


  0%|          | 0/18 [00:00<?, ?it/s]

...

T: algo_first, L: 10, Loss: 3.92188
T: algo_first, L: 11, Loss: 3.65625
T: algo_first, L: 12, Loss: 3.98438
T: algo_first, L: 13, Loss: 3.8125
T: algo_first, L: 14, Loss: 3.57812
T: algo_first, L: 15, Loss: 3.10938
T: algo_first, L: 16, Loss: 3.25
T: algo_first, L: 17, Loss: 2.92188
T: algo_first, L: 18, Loss: 2.57812
T: algo_first, L: 19, Loss: 2.76562
T: algo_first, L: 20, Loss: 2.8125
T: algo_first, L: 21, Loss: 2.8125
T: algo_first, L: 22, Loss: 2.82812
T: algo_first, L: 23, Loss: 3.09375
T: algo_first, L: 24, Loss: 3.07812
T: algo_first, L: 25, Loss: 2.98438
T: algo_first, L: 26, Loss: 2.98438
T: algo_first, L: 27, Loss: 2.98438
T: algo_sum,  zero loss: 5.25


  0%|          | 0/18 [00:00<?, ?it/s]

T: algo_sum, L: 10, Loss: 5.4375
T: algo_sum, L: 11, Loss: 4.9375
T: algo_sum, L: 12, Loss: 4.84375
T: algo_sum, L: 13, Loss: 4.84375
T: algo_sum, L: 14, Loss: 4.71875
T: algo_sum, L: 15, Loss: 4.84375
T: algo_sum, L: 16, Loss: 4.71875
T: algo_sum, L: 17, Loss: 4.78125
T: algo_sum, L: 18, Loss: 4.78125
T: algo_sum, L: 19, Loss: 4.875
T: algo_sum, L: 20, Loss: 4.8125
T: algo_sum, L: 21, Loss: 4.84375
T: algo_sum, L: 22, Loss: 4.96875
T: algo_sum, L: 23, Loss: 5.03125
T: algo_sum, L: 24, Loss: 5.03125
T: algo_sum, L: 25, Loss: 5
T: algo_sum, L: 26, Loss: 5.0625
T: algo_sum, L: 27, Loss: 5.0625
T: algo_most_common,  zero loss: 2.92188


  0%|          | 0/18 [00:00<?, ?it/s]

T: algo_most_common, L: 10, Loss: 3.71875
T: algo_most_common, L: 11, Loss: 2.95312
T: algo_most_common, L: 12, Loss: 2.67188
T: algo_most_common, L: 13, Loss: 2.78125
T: algo_most_common, L: 14, Loss: 2.79688
T: algo_most_common, L: 15, Loss: 2.89062
T: algo_most_common, L: 16, Loss: 2.9375
T: algo_most_common, L: 17, Loss: 2.70312
T: algo_most_common, L: 18, Loss: 2.79688
T: algo_most_common, L: 19, Loss: 2.79688
T: algo_most_common, L: 20, Loss: 2.84375
T: algo_most_common, L: 21, Loss: 2.95312
T: algo_most_common, L: 22, Loss: 2.95312
T: algo_most_common, L: 23, Loss: 3.0625
T: algo_most_common, L: 24, Loss: 2.98438
T: algo_most_common, L: 25, Loss: 2.96875
T: algo_most_common, L: 26, Loss: 2.92188
T: algo_most_common, L: 27, Loss: 2.9375


In [16]:

tvs = {}

task_names = [
    ("en_es", 18),
    ("en_fr", 18),
    ("en_it", 18),
    ("en_de", 18),
    ("en_ru", 18),
    ("es_en", 18),
    ("fr_en", 18),
    ("it_en", 20),
    ("antonyms", 18),
    ("plural_singular", 18),
    ("present_simple_past_simple", 20),
    ("country_capital", 18),
    ("algo_first", 18),
    ("location_country", 21)
]


for task, layer in tqdm(task_names):

    pairs = list(tasks[task].items())

    runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_few_shots-1, max_seq_len=max_seq_len, seed=10)

    tokenized = runner.get_tokens(runner.train_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, resids = get_resids_call(inputs)
    
    tokenized = runner.get_tokens(runner.eval_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    tokens = tokenized["input_ids"]

    logits = llama(inputs)
    

    loss = logprob_loss(
        logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
    )

    print(
        f"T: {task},  zero loss: {loss}"
    )
        
    # tv = get_tv(resids[layer].value.unwrap("batch", "seq", "embedding"), train_tokens, shift = 1 if task.startswith("algo") else 0)
    tv = get_tv(resids[layer].value.unwrap("batch", "seq", "embedding"), train_tokens, shift = 0)
    tv = tv.astype('bfloat16')

    add_act = make_act_adder(llama, tv, tokens, layer, length=1, shift= 0)

    logits = add_act(inputs)

    loss = logprob_loss(
        logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
    )

    print(
        f"T: {task}, L: {layer}, Loss: {loss}"  
    )

    tvs[task] = tv

  0%|          | 0/14 [00:00<?, ?it/s]

T: en_es,  zero loss: 14.625
T: en_es, L: 18, Loss: 4.65625
T: en_fr,  zero loss: 12.625
T: en_fr, L: 18, Loss: 5.6875
T: en_it,  zero loss: 15.375
T: en_it, L: 18, Loss: 3.0625
T: en_de,  zero loss: 12.125
T: en_de, L: 18, Loss: 6.9375
T: en_ru,  zero loss: 16.125
T: en_ru, L: 18, Loss: 5.8125
T: es_en,  zero loss: 8.5
T: es_en, L: 18, Loss: 1.71094
T: fr_en,  zero loss: 7.3125
T: fr_en, L: 18, Loss: 2.45312
T: it_en,  zero loss: 8.0625
T: it_en, L: 20, Loss: 4.25
T: antonyms,  zero loss: 5.96875
T: antonyms, L: 18, Loss: 2.29688
T: plural_singular,  zero loss: 2.53125
T: plural_singular, L: 18, Loss: 1.53906
T: present_simple_past_simple,  zero loss: 2.48438
T: present_simple_past_simple, L: 20, Loss: 0.773438
T: country_capital,  zero loss: 6.40625
T: country_capital, L: 18, Loss: 3.73438
T: algo_first,  zero loss: 3.20312
T: algo_first, L: 18, Loss: 2.57812
T: location_country,  zero loss: 5.625
T: location_country, L: 21, Loss: 3.67188


In [19]:
from micrlhf.utils.vector_storage import save_and_upload_vector

for task, layer in task_names:
    tv = tvs[task]
    save_and_upload_vector(f"task_vectors/{task}:{layer}.npz", tv)

In [297]:
for task, r in results.items():
    print(f"{task}: w/o: {r[0]}, w/: {min(r[1:])}, layer: {r.index(min(r[1:])) + 9}")

location_continent: w/o: 7.96875, w/: 1.91406, layer: 25
football_player_position: w/o: 13.125, w/: 4.375, layer: 22
location_religion: w/o: 7.875, w/: 1.55469, layer: 27
location_language: w/o: 7.9375, w/: 1.82812, layer: 21
person_profession: w/o: 11.3125, w/: 3.53125, layer: 24
location_country: w/o: 5.625, w/: 3.67188, layer: 21
country_capital: w/o: 6.40625, w/: 3.73438, layer: 18
person_language: w/o: 7.4375, w/: 1.28906, layer: 21
singular_plural: w/o: 1.96875, w/: 0.910156, layer: 21
present_simple_past_simple: w/o: 2.48438, w/: 0.773438, layer: 20
antonyms: w/o: 5.96875, w/: 2.29688, layer: 18
plural_singular: w/o: 2.53125, w/: 1.53906, layer: 18
present_simple_past_perfect: w/o: 4.65625, w/: 2.375, layer: 21
present_simple_gerund: w/o: 3.51562, w/: 1.17188, layer: 21
en_it: w/o: 15.375, w/: 3.0625, layer: 18
it_en: w/o: 8.0625, w/: 3.8125, layer: 18
en_ru: w/o: 16.125, w/: 3.76562, layer: 21
en_fr: w/o: 12.625, w/: 5.6875, layer: 18
en_es: w/o: 14.625, w/: 4.65625, layer: 18


In [282]:
for task in tqdm(task_names):
    # results[task] = []

    pairs = list(tasks[task].items())

    runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_few_shots-1, max_seq_len=max_seq_len, seed=10)

    tokenized = runner.get_tokens(runner.eval_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    tokens = tokenized["input_ids"]
    logits = llama(inputs)
    

    loss = logprob_loss(
        logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
    )

    print(
        f"T: {task},  Loss: {loss}"
    )

  0%|          | 0/6 [00:00<?, ?it/s]

T: algo_max,  Loss: 3.0625
T: algo_min,  Loss: 3
T: algo_last,  Loss: 3.39062
T: algo_first,  Loss: 3.20312
T: algo_sum,  Loss: 5.25
T: algo_most_common,  Loss: 2.92188


In [283]:
for task in tqdm(task_names):
    # results[task] = []

    pairs = list(tasks[task].items())

    runner = ICLRunner(task, pairs, batch_size=batch_size, n_shot=n_few_shots-1, max_seq_len=max_seq_len, seed=10)

    tokenized = runner.get_tokens(runner.train_pairs, tokenizer)
    inputs = tokenized_to_inputs(**tokenized)
    tokens = tokenized["input_ids"]
    logits = llama(inputs)
    

    loss = logprob_loss(
        logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=2
    )

    print(
        f"T: {task},  Loss: {loss}"
    )

  0%|          | 0/6 [00:00<?, ?it/s]

T: algo_max,  Loss: 1.82031
T: algo_min,  Loss: 2.45312
T: algo_last,  Loss: 0.0549316
T: algo_first,  Loss: 1.6875
T: algo_sum,  Loss: 4.09375
T: algo_most_common,  Loss: 3.04688


In [263]:
runner.get_prompt(runner.train_pairs[0])

'<|user|>\nFollow the pattern:\nmovement -> movimiento\nnow -> ahora\nthrow -> lanzar\nact -> actuar\ninteresting -> interesante\nor -> o\nreason -> razón\ndaughter -> hija\nwindow -> ventana\nsense -> sentido\nmain -> principal\nwar -> guerra\nthen -> entonces\nhot -> caliente\ndifficult -> difícil\nreach -> alcanzar\nstate -> expresar\nmoment -> momento\nreturn -> regreso'

In [264]:
tokenized = runner.get_tokens(runner.train_pairs, tokenizer)
inputs = tokenized_to_inputs(**tokenized)
tokens = tokenized["input_ids"]

logits, resids = get_resids_call(inputs)


tv = get_tv(resids[layer].value.unwrap("batch", "seq", "embedding"), tokens, shift = 1 if task.startswith("algo") else 0)

In [265]:
logprob_loss(
    logits.unwrap("batch", "seq", "vocabulary"), tokens, n_first=1, shift=1 if task.startswith("algo") else 0
)

<jax.Array(1.375, dtype=bfloat16)>

In [275]:
from task_vector_utils import make_act_adder

tokenized = runner.get_tokens(runner.eval_pairs, tokenizer)
inputs = tokenized_to_inputs(**tokenized)
tokens = tokenized["input_ids"]

add_act = make_act_adder(llama, tv * 0, tokens, layer, length=1, shift=1 if task.startswith("algo") else 0)

In [276]:
logits = add_act(inputs)

In [269]:
positions = jnp.argwhere(tokens == 1599)[:, -1] 

pred = jnp.take_along_axis(logits.unwrap("batch", "seq", "vocabulary"), positions[:, None, None], axis=1).squeeze().argmax(axis=-1)

In [270]:
pred

# jax.Array int32(16,) [≥491, ≤27_754] nonzero:16
  Array([ 7845,  3585, 10298,  2931, 27754,   491,  7623,  2317,  4847,
         14502,  4783,  5253,  9883,  9324,  1284,  6501], dtype=int32)

In [271]:
tokenizer.decode(pred)

'sit mention movement character argue by picture stand player treatment father amount sister strength find dark'

In [247]:
tokens[:, 1:]

array([[32000, 32000, 32000, ..., 29871, 29947, 29896],
       [32000, 32000, 32000, ..., 29871, 29929, 29929],
       [32000, 32000, 32000, ..., 29871, 29929, 29947],
       ...,
       [32000,     1, 32010, ..., 29871, 29947, 29929],
       [32000, 32000, 32000, ..., 29871, 29929, 29941],
       [32000, 32000, 32000, ..., 29871, 29896, 29906]])

In [277]:
logprob_loss(
    logits.unwrap("batch", "seq", "vocabulary"), tokens, shift=1 if task.startswith("algo") else 0, n_first=1
)

<jax.Array(10.0625, dtype=bfloat16)>

In [16]:
tvs = []

for task in tqdm(task_names):
    shot_logprobs_orig = [[] for _ in range(2)]
    shot_logprobs_added = [[] for _ in range(2)]
    shot_logprobs_zero = [[] for _ in range(2)]
    for seed in trange(n_seeds):
        pairs = tasks[task]
        pairs = [list(x) for x in pairs.items()]
        dataset = ICLDataset(pairs, size=batch_size, n_prepended=n_few_shots, bidirectional=False, seed=seed, prepend_space=task.startswith("algo"))

        tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
        inputs = tokenized_to_inputs(
            **tokenized
        )
        
        logits, resids = get_resids_call(inputs)
    
        tokens = tokenized["input_ids"]

        shot_logprobs_orig[0].append(
            get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True, extra_space=task.startswith("algo"))
        )

        shot_logprobs_orig[1].append(
            shot_logprobs_orig[0][-1] == 0.
        )

        mask = inputs.tokens == 1599
        mask = mask.unwrap("batch", "seq")

        tv = resids[layer].value.unwrap("batch", "seq", "embedding")[
            mask
        ]

        tv = tv.mean(
            axis=0
        )

        tvs.append(tv)

        print(
            tv.mean(), tv.std()
        )

        act_add = add_vector(
            llama, tv, layer, scale=2.0, position="last"
        )

        pairs = tasks[task]
        pairs = [list(x) for x in pairs.items()]
        dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=seed+1, prepend_space=task.startswith("algo"))


        print(
            dataset.prompts, dataset.completions
        )

        tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
        inputs = tokenized_to_inputs(
            **tokenized
        )

        logits = act_add(inputs)

        tokens = tokenized["input_ids"]

        shot_logprobs_added[0].append(
            get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True, extra_space=task.startswith("algo"))
        )

        shot_logprobs_added[1].append(
            shot_logprobs_added[0][-1] == 0.
        )

        logits, _ = get_resids_call(inputs)
        
        shot_logprobs_zero[0].append(
            get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True, extra_space=task.startswith("algo"))
        )

        shot_logprobs_zero[1].append(
            shot_logprobs_zero[0][-1] == 0.
        )
        
    print(f"orig: {shot_logprobs_orig}")
    print(f"zero: {shot_logprobs_zero}")
    print(f"added: {shot_logprobs_added}")


    shot_logprobs_orig = [list(map(np.mean, x)) for x in shot_logprobs_orig]
    shot_logprobs_zero = [list(map(np.mean, x)) for x in shot_logprobs_zero]
    shot_logprobs_added = [list(map(np.mean, x)) for x in shot_logprobs_added]


print(f"orig: {shot_logprobs_orig}")
print(f"zero: {shot_logprobs_zero}")
print(f"added: {shot_logprobs_added}")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [64]:
a

  0%|          | 0/1 [00:00<?, ?it/s]

-0.0090332 0.875
along window information over field current better power message reduce behavior place security hell cell media club live point throw
lungo fin inform sop campo att meg pot mess rid comport lu sic infer cell media club abit punto get
-0.00256348 0.898438
along window information over field current better power message reduce behavior place security hel cell media club live point throw
lungo fin inform sop campo att meg pot mess rid comport lu sic infer cell media club abit punto get
-0.00260925 0.957031
along window inform over campo current migli pot mess rid behavior place security h cell media club v punto throw
lungo fin inform sop campo att meg pot mess rid comport lu sic infer cell media club abit punto get
-0.00460815 1.09375
along fen inform over campo current migli pot mess rid comport pl seg h cell media club v punto throw
lungo fin inform sop campo att meg pot mess rid comport lu sic infer cell media club abit punto get
-0.00552368 1.17188


100%|██████████| 1/1 [00:22<00:00, 22.50s/it]

along window inform over campo current migli pot mess rid behavior place sic h cell media club v punto throw
lungo fin inform sop campo att meg pot mess rid comport lu sic infer cell media club abit punto get
orig: [[Array(-1.46094, dtype=bfloat16)], [Array(0.8, dtype=float32)]]
added: [[Array(-4.96875, dtype=bfloat16), Array(-4.75, dtype=bfloat16), Array(-1.53125, dtype=bfloat16), Array(-1.40625, dtype=bfloat16), Array(-1.72656, dtype=bfloat16)], [Array(0.15, dtype=float32), Array(0.15, dtype=float32), Array(0.45000002, dtype=float32), Array(0.6, dtype=float32), Array(0.55, dtype=float32)]]


In [61]:
shot_logprobs_added

[[<jax.Array(-4.40625, dtype=bfloat16)>,
  <jax.Array(-4.4375, dtype=bfloat16)>,
  <jax.Array(-1.30469, dtype=bfloat16)>,
  <jax.Array(-1.21875, dtype=bfloat16)>,
  <jax.Array(-1.51562, dtype=bfloat16)>],
 [<jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0.25, dtype=float32)>,
  <jax.Array(0.35, dtype=float32)>,
  <jax.Array(0.35, dtype=float32)>]]

In [27]:

from micrlhf.utils.load_sae import get_sae
sae = get_sae(layer, 6)

--2024-05-23 18:36:35--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l18-test-run-6-1.01E-05/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/f057cb46f3d871ba03c66e707e3b3d8299322f36fa433862dc3fdca956715614?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1716748596&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNjc0ODU5Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvZjA1N2NiNDZm

In [28]:
from micrlhf.utils.ito import grad_pursuit

In [30]:
k = 20

weights, recon = grad_pursuit(tv, sae["W_dec"], k, pos_only=True)
w, i = jax.lax.top_k(jnp.abs(weights), k)

i

<jax.Array int32(20,) [≥291, ≤47_625] nonzero:20
  <Arrayviz rendering>
  Replicated across 4 TPU devices (click to expand)
>

In [46]:
jnp.linalg.norm(tvs[0] - recon)

<jax.Array(36.703175, dtype=float32)>

In [36]:
k = 5

[
    jax.lax.top_k(jnp.abs(grad_pursuit(x, sae["W_dec"], k, pos_only=True)[0]), k) for x in tvs
]

[[<Array([29.792599, 10.352382,  8.920501,  8.145139,  8.122905], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 30355, 45211], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([29.806067, 10.385991,  8.90197 ,  8.17028 ,  8.131155], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 30355], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([29.883137, 10.367022,  8.96897 ,  8.084253,  7.998138], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 30355], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.066097 ,  9.934102 ,  8.822496 ,  8.050564 ,  7.8920116],      dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 30355], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.23363  ,  9.876981 ,  8.7954235,  8.072471 ,  7.838931 ],      dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 30355], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.243702 ,  9.882975 ,  8.824527 ,  8.136261 ,  7.9271026],      dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 30355], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.247698,  9.898605,  8.814878,  8.135923,  7.857659], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 26986], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.279543,  9.898817,  8.800136,  8.145805,  8.018657], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 26986], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([26.278713,  9.908989,  8.835454,  8.160914,  8.048674], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 45211, 26986], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >],
 [<Array([30.394947, 10.208616,  9.004849,  8.210366,  8.165597], dtype=float32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >,
  <Array([34560, 40991, 24043, 26986, 45211], dtype=int32)
    <Arrayviz rendering>
    Replicated across 4 TPU devices (click to expand)
  >]]

In [27]:

task_names = [
    "en_ru"
]

layer = 18

for task in tqdm(task_names):
    shot_logprobs_orig = [[] for _ in range(2)]
    shot_logprobs_added = [[] for _ in range(2)]
    shot_logprobs_sae = [[] for _ in range(2)]

    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=n_few_shots, bidirectional=False, seed=10, prepend_space=task.startswith("algo"))

    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )
    
    logits, resids = get_resids_call(inputs)

    tokens = tokenized["input_ids"]

    shot_logprobs_orig[0].append(
        get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=False, extra_space=task.startswith("algo"))
    )

    shot_logprobs_orig[1].append(
        shot_logprobs_orig[0][-1] == 0.
    )

    mask = inputs.tokens == 1599
    mask = mask.unwrap("batch", "seq")

    tv = resids[layer].value.unwrap("batch", "seq", "embedding")[
        mask
    ]

    tv = tv.mean(
        axis=0
    )

    print(
        tv.mean(), tv.std()
    )

    act_add = add_vector(
        llama, tv, layer, scale=2.0, position="last"
    )

    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=11, prepend_space=task.startswith("algo"))


    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )

    logits = act_add(inputs)

    tokens = tokenized["input_ids"]

    shot_logprobs_added[0].append(
        get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True, extra_space=task.startswith("algo"))
    )

    shot_logprobs_added[1].append(
        shot_logprobs_added[0][-1] == 0.
    )

    for k in range(0, 40):
        weights, recon = grad_pursuit(tv * 2, sae["W_dec"], k, pos_only=True)

        act_add = add_vector(
            llama, recon.astype('bfloat16'), layer, scale=1.0, position="last"
        )

        logits = act_add(inputs)
        
        shot_logprobs_sae[0].append(
            get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True, extra_space=task.startswith("algo"))
        )

        shot_logprobs_sae[1].append(
            shot_logprobs_sae[0][-1] == 0.
        )
        
    # print(f"orig: {shot_logprobs_orig}")
    # print(f"sae: {shot_logprobs_sae}")
    # print(f"added: {shot_logprobs_added}")


    shot_logprobs_orig = [list(map(np.mean, x)) for x in shot_logprobs_orig]
    shot_logprobs_sae = [list(map(np.mean, x)) for x in shot_logprobs_sae]
    shot_logprobs_added = [list(map(np.mean, x)) for x in shot_logprobs_added]


print(f"orig: {shot_logprobs_orig}")
print(f"sae: {shot_logprobs_sae}")
print(f"added: {shot_logprobs_added}")

  0%|          | 0/1 [00:00<?, ?it/s]

-0.00294495 1.10156
об amb ре  пи пи вопро ка кни пи день ок кни пу  г ве kitchen  less  ме вто ко<|placeholder1|> мате ко<|placeholder1|> ви мо би bus ф е г  та е date пи проду тан су ме прода tennis мате<|placeholder1|> ми п би би ве пе amb<|placeholder1|> пи check проду з<|placeholder1|> bus price са
зав ско ри дере сви га вопро ка кни х день ок те команди ло г ос ку мо у мо а се ко сез мате ко от с го би авто ф пи г дере сто е да на проду та су сту прода т мате сез ми учи би би з ру ско сез пи реги проду з от авто це са
br amb rice trees p bur answer calendar books bread money wind not vac boat be aut kitchen ice less ice ph second hat season math hat vac viol mount bi bus fl food be trees table me month drink product dan soup chair sell tennis math season minutes engineer business bi winter p amb season pie check product * vac bus price sal
зав ско ри дере сви га вопро ка кни х день ок те команди ло г ос ку мо у мо а се ко сез мате ко от с го би авто ф пи г дере сто е да на проду 

In [28]:
shot_logprobs_sae

[[<jax.Array(-14.4375, dtype=bfloat16)>,
  <jax.Array(-14.125, dtype=bfloat16)>,
  <jax.Array(-7.84375, dtype=bfloat16)>,
  <jax.Array(-8.75, dtype=bfloat16)>,
  <jax.Array(-8.5, dtype=bfloat16)>,
  <jax.Array(-8.4375, dtype=bfloat16)>,
  <jax.Array(-9.5, dtype=bfloat16)>,
  <jax.Array(-9.8125, dtype=bfloat16)>,
  <jax.Array(-10.5, dtype=bfloat16)>,
  <jax.Array(-10.5, dtype=bfloat16)>,
  <jax.Array(-10.125, dtype=bfloat16)>,
  <jax.Array(-10.125, dtype=bfloat16)>,
  <jax.Array(-9.9375, dtype=bfloat16)>,
  <jax.Array(-9.9375, dtype=bfloat16)>,
  <jax.Array(-9.9375, dtype=bfloat16)>,
  <jax.Array(-7.1875, dtype=bfloat16)>,
  <jax.Array(-7.125, dtype=bfloat16)>,
  <jax.Array(-4.6875, dtype=bfloat16)>,
  <jax.Array(-4.8125, dtype=bfloat16)>,
  <jax.Array(-5, dtype=bfloat16)>,
  <jax.Array(-4.9375, dtype=bfloat16)>,
  <jax.Array(-4.90625, dtype=bfloat16)>,
  <jax.Array(-4.6875, dtype=bfloat16)>,
  <jax.Array(-4.6875, dtype=bfloat16)>,
  <jax.Array(-4.875, dtype=bfloat16)>,
  <jax.Array(-5.125, dtype=bfloat16)>,
  <jax.Array(-5.375, dtype=bfloat16)>,
  <jax.Array(-5.28125, dtype=bfloat16)>,
  <jax.Array(-5.40625, dtype=bfloat16)>,
  <jax.Array(-5.3125, dtype=bfloat16)>,
  <jax.Array(-5.375, dtype=bfloat16)>,
  <jax.Array(-3.73438, dtype=bfloat16)>,
  <jax.Array(-4, dtype=bfloat16)>,
  <jax.Array(-4.03125, dtype=bfloat16)>,
  <jax.Array(-3.92188, dtype=bfloat16)>,
  <jax.Array(-3.82812, dtype=bfloat16)>,
  <jax.Array(-4.03125, dtype=bfloat16)>,
  <jax.Array(-4.125, dtype=bfloat16)>,
  <jax.Array(-4.21875, dtype=bfloat16)>,
  <jax.Array(-4.46875, dtype=bfloat16)>],
 [<jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.046875, dtype=float32)>,
  <jax.Array(0.03125, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.03125, dtype=float32)>,
  <jax.Array(0., dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.015625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.0625, dtype=float32)>,
  <jax.Array(0.03125, dtype=float32)>,
  <jax.Array(0.03125, dtype=float32)>]]

In [40]:
shot_logprobs_sae[0][18]

<jax.Array(-4.8125, dtype=bfloat16)>

In [41]:
k = 18

weights, recon = grad_pursuit(tv, sae["W_dec"], k, pos_only=True)
w, i = jax.lax.top_k(jnp.abs(weights), k)

i

<jax.Array int32(18,) [≥764, ≤47_625] nonzero:18
  <Arrayviz rendering>
  Replicated across 4 TPU devices (click to expand)
>

In [38]:
w

<jax.Array float32(18,) ≈7.4 ±3.2 [≥4.4, ≤1.9e+01] nonzero:18
  <Arrayviz rendering>
  Replicated across 4 TPU devices (click to expand)
>

In [ ]:
#37312

In [44]:
pairs = tasks[task]
pairs = [list(x) for x in pairs.items()]
dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=101, prepend_space=task.startswith("algo"))


tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
inputs = tokenized_to_inputs(
    **tokenized
)

# weights, recon = grad_pursuit(tv * 2, sae["W_dec"], k, pos_only=True)

recon = sae["W_dec"][27215] * 20

act_add = add_vector(
    llama, recon.astype('bfloat16'), layer, scale=1.0, position="last"
)

logits = act_add(inputs)

get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=True).mean()

less snow inst train st car t window concert the air new guitar vis table unique take a sal there actor sal higher emer per sal unique inst feu football library light river wine higher any time flight sal football milk less river су concert sa air higher service sal k summer milk meat football no name name milk tor ly sal plan less
у с университе мет сту ма вра ок кон те а ве ги ви сто с га су са по филь пи не ско ба сотруд день университе роман т би со ре ви не от время по профес т га у ре де кон са само не ус день пе ве мо г спо на час город мо би га це п у


<jax.Array(-8.625, dtype=bfloat16)>

In [26]:
dataset.completions

['stärke',
 'papier',
 'lied',
 'arzt',
 'atom',
 'gans',
 'auto',
 'experiment',
 'freund',
 'zucker',
 'bild',
 'stern',
 'insekt',
 'vergangenheit',
 'regisseur',
 'wissen',
 'film',
 'lärm',
 'brücke',
 'theater',
 'affe',
 'tatsache',
 'forum',
 'bleistift',
 'sonne',
 'regel',
 'sirene',
 'freund',
 'meer',
 'stern',
 'blatt',
 'foto',
 'klang',
 'hügel',
 'haus',
 'heute',
 'gas',
 'weisheit',
 'weltraum',
 'motor',
 'information',
 'planet',
 'daten',
 'jahrtausend',
 'armbanduhr',
 'atom',
 'finsternis',
 'regen',
 'abend',
 'nacht',
 'huhn',
 'lärm',
 'stift',
 'garnele',
 'insel',
 'programm',
 'alligator',
 'eis',
 'mond',
 'substanz',
 'daten',
 'tee',
 'wald',
 'tag']

In [17]:
def prepare_inputs(dataset: ICLDataset):
    tokenized = tokenizer.batch_encode_plus([prompt.format(x) for x in dataset.prompts], padding="longest", max_length=max_seq_len, truncation=True, return_tensors="np")
    inputs = tokenized_to_inputs(
        **tokenized
    )
    
    tokens = tokenized["input_ids"]

    return inputs, tokens

In [18]:
task_names = list(tasks.keys())
task_names = [x for x in task_names if x.startswith("algo")]
# n_few_shots, batch_size, max_seq_len = 64, 64, 512
n_few_shots, batch_size, max_seq_len = 20, 64, 256

In [24]:
results = {}

for task in tqdm(task_names):
    results[task] = []
    
    pairs = tasks[task]
    pairs = [list(x) for x in pairs.items()]
    dataset = ICLDataset(pairs, size=batch_size, n_prepended=n_few_shots, bidirectional=False, seed=10, prepend_space=task.startswith("algo"))

    clean_inputs, clean_tokens = prepare_inputs(dataset)

    _, resids = get_resids_call(clean_inputs)

    mask = clean_inputs.tokens == 1599
    mask = mask.unwrap("batch", "seq")

    if task.startswith("algo"):
        mask = jnp.roll(mask, 1, axis=1)

    dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=11, prepend_space=task.startswith("algo"))

    add_inputs, add_tokens = prepare_inputs(dataset)

    for layer in trange(10, 22):
        tv = resids[layer].value.unwrap("batch", "seq", "embedding")[mask]
        tv = tv.mean(axis=0)
        tv = tv.astype('bfloat16')

        act_add = add_vector(
            llama, tv, layer, scale=2.0, position="last"
        )

        logits = act_add(add_inputs)

        diff = get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=False, extra_space=task.startswith("algo"))

        results[task].append(diff.mean())
        
        

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [25]:
for k in results:
    print(
        k, np.argmax([float(x) for x in results[k]]) + 10, max(results[k])
    )

algo_max 21 -0.515625
algo_min 21 -0.945312
algo_last 20 -0.162109
algo_first 13 -0.828125
algo_sum 17 -1.49219
algo_most_common 14 -1.15625


In [31]:
for k in results:
    print(
        k, np.argmax([float(x) for x in results[k]]) + 10, max(results[k])
    )

location_continent 21 -1.09375
football_player_position 21 -3.76562
location_religion 21 -1.09375
location_language 20 -0.457031
person_profession 21 -1.30469
location_country 21 -1.44531
country_capital 18 -1.25781
person_language 18 -0.287109
singular_plural 21 -0.261719
present_simple_past_simple 20 -0.341797
antonyms 14 -0.75
plural_singular 20 -0.133789
present_simple_past_perfect 18 -1.25781
present_simple_gerund 20 -0.171875
en_it 18 -1.29688
it_en 14 -1.75781
en_fr 18 -1.39062
en_es 18 -1.21875
fr_en 17 -1.25781
es_en 17 -1.09375


In [22]:
# for task in tqdm(task_names):

task = "algo_last"

results[task] = []

pairs = tasks[task]
pairs = [list(x) for x in pairs.items()]
dataset = ICLDataset(pairs, size=batch_size, n_prepended=n_few_shots, bidirectional=False, seed=10, prepend_space=task.startswith("algo"))

clean_inputs, clean_tokens = prepare_inputs(dataset)

_, resids = get_resids_call(clean_inputs)

mask = clean_inputs.tokens == 1599
mask = mask.unwrap("batch", "seq")

mask = jnp.roll(mask, 1, axis=1)

dataset = ICLDataset(pairs, size=batch_size, n_prepended=0, bidirectional=False, seed=11, prepend_space=task.startswith("algo"))

add_inputs, add_tokens = prepare_inputs(dataset)

for layer in trange(10, 22):
    tv = resids[layer].value.unwrap("batch", "seq", "embedding")[mask]
    tv = tv.mean(axis=0)
    tv = tv.astype('bfloat16')

    act_add = add_vector(
        llama, tv, layer, scale=2.0, position="last"
    )

    logits = act_add(add_inputs)

    diff = get_logprob_diff(logits.unwrap("batch", "seq", "vocabulary"), dataset.completions, print_results=False, extra_space=task.startswith("algo"))

    results[task].append(diff.mean())

  0%|          | 0/12 [00:00<?, ?it/s]

In [23]:
results[task]

[<jax.Array(-0.898438, dtype=bfloat16)>,
 <jax.Array(-0.835938, dtype=bfloat16)>,
 <jax.Array(-0.757812, dtype=bfloat16)>,
 <jax.Array(-0.308594, dtype=bfloat16)>,
 <jax.Array(-0.269531, dtype=bfloat16)>,
 <jax.Array(-0.243164, dtype=bfloat16)>,
 <jax.Array(-0.287109, dtype=bfloat16)>,
 <jax.Array(-0.249023, dtype=bfloat16)>,
 <jax.Array(-0.173828, dtype=bfloat16)>,
 <jax.Array(-0.1875, dtype=bfloat16)>,
 <jax.Array(-0.162109, dtype=bfloat16)>,
 <jax.Array(-0.263672, dtype=bfloat16)>]

In [10]:
from task_vector_utils import logprob_loss, ICLRunner

NameError: name 'batch_size' is not defined